This script is to pull from the existing Heliophysics vocabularies to create a database of all terms, whether they are defined, the number of times they are defined, and any gaps

compiling relevant space weather and helio relevant vocabularies [here](https://docs.google.com/spreadsheets/d/15D5tm6TchdZXfwjnDpJMh8ICCsU4C3qGM-amX9YjZkM/edit?usp=sharing)

We will not attempt to bring in all glossaries in this project. The subset we will begin with are: 
- Unified Astronomy Thesaurus
- AGU index terms
- NASA Heliophysics Vocabulary
- SPASE

Stretch glossaries will include: 
- NOAA Space Weather
- ESA Space Weather Glossary
- (Maybe) a very small subset of the AMS Glossary of Meteorology






In [30]:
import os, json
import numpy as np
import pandas as pd

import re

In [31]:
directory_vocabs = 'data/'


In [32]:
# Read in existing solar-related terms from UAT
pd_UAT = pd.read_csv(os.path.join(directory_vocabs,'UAT_Solar-related-concepts.csv'))
pd_UAT

,level 1,level 2,level 3,level 4,level 5,level 6,level 7,level 8,level 9,level 10,level 11
0,Observational astronomy,Astronomical instrumentation,Observatories,Solar observatories,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Observational astronomy,Astronomical instrumentation,Solar instruments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Observational astronomy,Astronomical instrumentation,Stellar tracking devices,Heliostats,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Observational astronomy,Astronomical instrumentation,Telescopes,Optical telescopes,Solar optical telescopes,NaN,NaN,NaN,NaN,NaN,NaN
4,Observational astronomy,Astronomical instrumentation,Telescopes,Radio telescopes,Solar radio telescopes,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
439,Solar system astronomy,Solar system,Space weather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,Solar system astronomy,Solar system,The Moon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,Solar system astronomy,Solar system,The Sun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,Solar system astronomy,Solar system,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Transforming to a new dataframe that can be combined with the other glossaries
list_UAT_terms = []
for c in pd_UAT.columns:
    print(c)
    list_UAT_terms = list_UAT_terms + list(set(pd_UAT[c].values))
    
list_UAT_new = []
for val in list_UAT_terms:
    if isinstance(val,float):
#         list_UAT_terms.remove(val)
        continue
    list_UAT_new = list_UAT_new + [val]
list_UAT_terms = [val.lower() for val in list_UAT_new]

pd_UAT_terms = pd.DataFrame(columns=['term','definition'])
pd_UAT_terms['term'] = list_UAT_terms

pd_UAT_terms['source'] = np.tile('uat',(len(pd_UAT_terms),1))

level 1
level 2
level 3
level 4
level 5
level 6
level 7
level 8
level 9
level 10
level 11


In [34]:
list_UAT_terms

['solar physics',
 'solar system astronomy',
 'observational astronomy',
 'solar particle emission',
 'solar spectral irradiance',
 'solar evolution',
 'solar wind',
 'solar radius',
 'solar magnetic fields',
 'solar mass',
 'solar electromagnetic emission',
 'helioseismology',
 'solar system',
 'solar abundances',
 'solar interior',
 'planetary science',
 'earth-moon system',
 'solar atmosphere',
 'cno anomaly',
 'solar activity',
 'lunar science',
 'sunspots',
 'solar flares',
 'solar radiation',
 'solar oscillations',
 'solar granulation',
 'solar motion',
 'solar surface',
 'astronomical techniques',
 'solar faculae',
 'astronomical instrumentation',
 'solar rotation',
 'natural satellite atmospheres',
 'antapex',
 'solar prominences',
 'lunar phase',
 'kuiper belt',
 'comet origins',
 'optical observation',
 'solar corona',
 'solar convective zone',
 'solar flares',
 'delta sunspots',
 'lunar origin',
 'planetary atmospheres',
 'planetary climates',
 'meteoroids',
 'planetary-disk

In [35]:
pd_UAT_terms

,term,definition,source
0,solar physics,NaN,uat
1,solar system astronomy,NaN,uat
2,observational astronomy,NaN,uat
3,solar particle emission,NaN,uat
4,solar spectral irradiance,NaN,uat
...,...,...,...
423,kreutz group,NaN,uat
424,meyer group,NaN,uat
425,gegenschein,NaN,uat
426,oort cloud objects,NaN,uat


In [36]:
# Read in AGU Index Terms
pd_agu = pd.read_excel(os.path.join(directory_vocabs,'agu-index-terms.xlsx'))
pd_agu

# Pull out only the section terms we are interested in
#  - Informatics (1900)
#  - Interplanetary Physics (2100)
#  - Ionosphere (2400)
#  - Magnetosphere (2700)
#  - Space Weather (2101, 2788, 7900, 4305)
#  - Extreme events (1817, 3235, 4313)
#  - Nonlinear Geophysics (3200, 4307, 6944, 7839, 4400)
#  - Radio Science (6900)
#  - Solar Physics, Astrophysics, and Astronomy (7500)
#  - Space Plasma Physics (7800)
#  - (maybe) Mathematical Geophysics (3200)
#  - (maybe) Natural Hazards (4300)

# NOTE: Important to maintain the hierarchy
#  Establish structure that maintains the categories above defines the terms within them (e.g., Space Weather: Geomagnetically induced currents)

# NOTE: removing XX99 terms because they are always 'General or miscellaneous'

# Remove duplicate terms


,Code,Description
0,200,GEOHEALTH
1,205,Archaeological Geology
2,210,Coal Geology
3,215,Economic geology
4,216,Engineering geology
...,...,...
1313,9805,Instruments useful in three or more fields
1314,9810,New fields (not classifiable under other headi...
1315,9815,Notices and announcements
1316,9820,Techniques applicable in three or more fields


In [37]:

for r in range(len(pd_agu)): 
    if ~( (pd_agu['Code'][r] >= 1900) & (pd_agu['Code'][r] < 1999) |
         (pd_agu['Code'][r] >= 2100) & (pd_agu['Code'][r] < 2199) |
         (pd_agu['Code'][r] >= 2400) & (pd_agu['Code'][r] < 2499) |
         (pd_agu['Code'][r] >= 2700) & (pd_agu['Code'][r] < 2799) |
         (pd_agu['Code'][r] >= 3200) & (pd_agu['Code'][r] < 3299) |
         (pd_agu['Code'][r] >= 4300) & (pd_agu['Code'][r] < 4399) |
         (pd_agu['Code'][r] >= 6900) & (pd_agu['Code'][r] < 6999) |
         (pd_agu['Code'][r] >= 7500) & (pd_agu['Code'][r] < 7599) |
         (pd_agu['Code'][r] >= 7800) & (pd_agu['Code'][r] < 7899) ):
#         print('Code = {} --> Term = {}'.format(pd_agu['Code'][r],pd_agu['Description'][r]))
        pd_agu = pd_agu.drop([r])

for r in pd_agu.index:
    if '(' in pd_agu['Description'][r]:
        print('prior = {}'.format(pd_agu['Description'][r]))
        pd_agu['Description'][r] = pd_agu['Description'][r][0:pd_agu['Description'][r].find('(')-1]
        print('post = {}'.format(pd_agu['Description'][r]))

pd_agu

prior = Decision analysis (4324, 6309)
post = Decision analysis
prior = Forecasting (2722, 4315, 7924)
post = Forecasting
prior = Machine learning (0555)
post = Machine learning
prior = Modeling (0466, 0545, 0798, 1847, 4255, 4316)
post = Modeling
prior = Real-time and responsive information delivery (4346)
post = Real-time and responsive information delivery
prior = Spatial analysis and representation (0500, 3252)
post = Spatial analysis and representation
prior = Statistical methods: Descriptive (4318)
post = Statistical methods: Descriptive
prior = Statistical methods: Inferential (4318)
post = Statistical methods: Inferential
prior = Temporal analysis and representation (1872, 3270, 4277, 4475)
post = Temporal analysis and representation
prior = Uncertainty (1873, 3275)
post = Uncertainty
prior = Visualization and portrayal (0530)
post = Visualization and portrayal
prior = Coronal mass ejections (4305, 7513)
post = Coronal mass ejections
prior = Discontinuities (7811)
post = Discon

<ipython-input-37-c96e85466c84>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_agu['Description'][r] = pd_agu['Description'][r][0:pd_agu['Description'][r].find('(')-1]


,Code,Description
432,1900,INFORMATICS
433,1902,Community modeling frameworks
434,1904,Community standards
435,1906,"Computational models, algorithms"
436,1908,Cyberinfrastructure
...,...,...
1154,7859,Transport processes
1155,7863,Turbulence
1156,7867,Wave/particle interactions
1157,7868,Wave/wave interactions


In [38]:
# Transforming into a new dataframe that can be combined with other glossaries
pd_agu_terms = pd.DataFrame(columns=['term','definition'])
pd_agu_terms['term'] = pd_agu['Description']
pd_agu_terms

pd_agu_terms['term'] = pd_agu_terms['term'].str.lower()

pd_agu_terms['source'] = np.tile('agu',(len(pd_agu_terms),1))
pd_agu_terms


,term,definition,source
432,informatics,NaN,agu
433,community modeling frameworks,NaN,agu
434,community standards,NaN,agu
435,"computational models, algorithms",NaN,agu
436,cyberinfrastructure,NaN,agu
...,...,...,...
1154,transport processes,NaN,agu
1155,turbulence,NaN,agu
1156,wave/particle interactions,NaN,agu
1157,wave/wave interactions,NaN,agu


In [39]:
os.path.exists(os.path.join(directory_vocabs,'spase-2.5.0-draft.json'))

True

In [40]:
# Read in SPASE
f = open(os.path.join(directory_vocabs,'spase-2.5.0-draft.json'))
json_spase = json.load(f)
f.close()
# json_spase
json_spase['dictionary']

# pd_spase = pd.read_json(os.path.join(directory_vocabs,'spase-2.5.0-draft.json'))
# pd_spase


{'Absorption': {'version': '2.5.0',
  'since': '1.3.5',
  'term': 'Absorption',
  'type': 'Item',
  'list': '',
  'element': '',
  'attributes': '',
  'definition': 'Decrease of radiant energy (relative to the background continuum spectrum).',
  'usedBy': [],
  'allowedValues': []},
 'AccessInformation': {'version': '2.5.0',
  'since': '1.0.0',
  'term': 'AccessInformation',
  'type': 'Container',
  'list': '',
  'element': '',
  'attributes': '',
  'definition': 'Attributes of the resource which pertain to how to accessing the resource, availability and storage format.',
  'usedBy': ['Catalog', 'DisplayData', 'Document', 'NumericalData'],
  'subElements': ['RepositoryID',
   'Availability',
   'AccessRights',
   'AccessURL',
   'Format',
   'Encoding',
   'DataExtent',
   'Acknowledgement'],
  'allowedValues': []},
 'AccessRights': {'version': '2.5.0',
  'since': '1.0.0',
  'term': 'AccessRights',
  'type': 'Enumeration',
  'list': 'AccessRights',
  'element': '',
  'attributes': '',


In [41]:
json_spase['dictionary']['AccessInformation']

{'version': '2.5.0',
 'since': '1.0.0',
 'term': 'AccessInformation',
 'type': 'Container',
 'list': '',
 'element': '',
 'attributes': '',
 'definition': 'Attributes of the resource which pertain to how to accessing the resource, availability and storage format.',
 'usedBy': ['Catalog', 'DisplayData', 'Document', 'NumericalData'],
 'subElements': ['RepositoryID',
  'Availability',
  'AccessRights',
  'AccessURL',
  'Format',
  'Encoding',
  'DataExtent',
  'Acknowledgement'],
 'allowedValues': []}

In [42]:
for key in json_spase['dictionary']:
    if 

SyntaxError: invalid syntax (<ipython-input-42-8bf50cc6c720>, line 2)

In [43]:

term_list_spase = []
definition_list_spase = []
type_list_spase = []
for key in json_spase['dictionary']:
#     print(key, '->', json_spase['dictionary'][key])
    term_list_spase.append(key)
    definition_list_spase.append(json_spase['dictionary'][key]['definition'])
    type_list_spase.append(json_spase['dictionary'][key]['type'])
        
pd_spase_terms = pd.DataFrame({'term':term_list_spase,'definition':definition_list_spase,'type':type_list_spase})

# separate words in SPASE
for t in range(len(pd_spase_terms['term'])):
    
#     print(re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', spase_pd['term'][t]))
    pd_spase_terms['term'][t] = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', pd_spase_terms['term'][t])
    
pd_spase_terms

pd_spase_terms['term'] = pd_spase_terms['term'].str.lower()
pd_spase_terms['source'] = np.tile('spase',(len(pd_spase_terms),1))

# Drop all types other than Item and Enumeration
mask_full = ( (pd_spase_terms['type']=='Container') | (pd_spase_terms['type']=='Text') | (pd_spase_terms['type']=='Numeric') | (pd_spase_terms['type']=='Duration') )
pd_spase_terms = pd_spase_terms.drop(pd_spase_terms.index[mask_full])
pd_spase_terms = pd_spase_terms.drop(columns=['type'])

pd_spase_terms

,term,definition,source
0,absorption,Decrease of radiant energy (relative to the ba...,spase
2,access rights,Permissions granted or denied by the host of a...,spase
4,ac electric field,Alternating electric field component of a wave.,spase
6,ac magnetic field,Alternating magnetic field component of a wave.,spase
7,active,Exerting an influence or producing a change or...,spase
...,...,...,...
737,white light,Photons with a wavelength in the visible range...,spase
738,white paper,An authoritative report giving information or ...,spase
739,xml,eXtensible Mark-up Language (XML). A structure...,spase
740,x rays,Photons with a wavelength range: 0.001 <= x < ...,spase


In [44]:
# Read in NASA Heliophysics Vocabulary 
file_NASAhelio = 'NASA Heliophysics Vocabulary.xlsx'
pd_NASAhelio_terms = pd.read_excel(os.path.join(directory_vocabs,file_NASAhelio))
pd_NASAhelio_terms
pd_NASAhelio_terms = pd_NASAhelio_terms.rename(columns={"Term": "term", "Definition":"definition"})
pd_NASAhelio_terms

pd_NASAhelio_terms['term'] = pd_NASAhelio_terms['term'].str.lower()
pd_NASAhelio_terms['source'] = np.tile('nasa',(len(pd_NASAhelio_terms),1))

pd_NASAhelio_terms

,term,definition,source
0,aurora,An aurora is a natural display of light in the...,nasa
1,chromosphere,The relatively thin layer of the solar atmosph...,nasa
2,corona,The sun's dynamic atmosphere is called the cor...,nasa
3,coronal mass ejection (cme),Not to be confused with the intense burst of l...,nasa
4,geomagnetic storm,"One of the most common forms of space weather,...",nasa
5,geomagnetically induced current (gic),The magnetic fluctuations caused by a geomagne...,nasa
6,heliosphere,"The heliosphere is our entire solar system, as...",nasa
7,interplanetary magnetic field (imf),As the solar wind flows outward from the sun a...,nasa
8,ionosphere,The ionosphere is a layer of Earth's atmospher...,nasa
9,magnetic field,A field of force generated by electrical curre...,nasa


In [45]:
# Compile terms from all glossaries and make a list of the common terms
frames = [pd_UAT_terms,pd_agu_terms,pd_spase_terms,pd_NASAhelio_terms]

pd_terms_total = pd.concat(frames)
pd_terms_total.reset_index(inplace=True)
pd_terms_total = pd_terms_total.drop(columns=['index'])
pd_terms_total


,term,definition,source
0,solar physics,NaN,uat
1,solar system astronomy,NaN,uat
2,observational astronomy,NaN,uat
3,solar particle emission,NaN,uat
4,solar spectral irradiance,NaN,uat
...,...,...,...
1313,solar maximum,The time during the 11-year solar cycle when t...,nasa
1314,solar minimum,The time during the 11-year solar cycle when t...,nasa
1315,solar wind,The constant stream of solar coronal material ...,nasa
1316,substorm,A release of magnetic energy that originates i...,nasa


In [46]:
pd_terms_total_unique = pd_terms_total.sort_index().groupby('term').filter(lambda group: len(group) == 1)
pd_terms_total_unique

,term,definition,source
0,solar physics,NaN,uat
1,solar system astronomy,NaN,uat
2,observational astronomy,NaN,uat
3,solar particle emission,NaN,uat
4,solar spectral irradiance,NaN,uat
...,...,...,...
1309,radiative zone,"In the radiative zone, energy from the core sl...",nasa
1311,solar energetic particles (sep),"During an eruptive event on the sun, the sun c...",nasa
1313,solar maximum,The time during the 11-year solar cycle when t...,nasa
1314,solar minimum,The time during the 11-year solar cycle when t...,nasa


In [47]:
pd_terms_total_nonunique = pd_terms_total.sort_index().groupby('term').filter(lambda group: len(group) > 1)
pd_terms_total_nonunique

,term,definition,source
6,solar wind,NaN,uat
8,solar magnetic fields,NaN,uat
11,helioseismology,NaN,uat
21,sunspots,NaN,uat
22,solar flares,NaN,uat
...,...,...,...
1308,radiation belts,"Two belts of radiation that surround Earth, al...",nasa
1310,solar cycle,The sun goes through 11-year variations or cyc...,nasa
1312,solar flares,A great burst of light and radiation due to th...,nasa
1315,solar wind,The constant stream of solar coronal material ...,nasa


In [ ]:
# TODO
# (DONE - June 14, 2022; RMM) Read in NASA Heliophysics Vocabulary 

# Compile terms from all glossaries, normalize the terms, and make a list of the common terms


# Read in NOAA Space Weather Glossary
# Read in ESA Space Weather Glossary
# Read in and subselect AMS Glossary of Meteorology


In [ ]:
# Read in NOAA Space Weather Glossary
pd_noaa # Need to read from HTML: https://www.swpc.noaa.gov/content/space-weather-glossary

In [49]:
pd_terms_total_nonunique = pd_terms_total_nonunique.drop_duplicates(subset=['term'])


In [50]:
pd_terms_total_nonunique

,term,definition,source
6,solar wind,NaN,uat
8,solar magnetic fields,NaN,uat
11,helioseismology,NaN,uat
21,sunspots,NaN,uat
22,solar flares,NaN,uat
...,...,...,...
733,aurora,An atmospheric phenomenon consisting of bands ...,spase
880,geomagnetic storm,A magnetospheric disturbance typically defined...,spase
989,magnetic field,A region of space near a magnetized body where...,spase
997,magnetosphere,The region of space above the atmosphere or su...,spase


In [51]:
# search for what is in the term list (non-unique)

pd_terms_total_nonunique.loc[pd_terms_total_nonunique['term'].str.contains('iono',case=False)]

,term,definition,source
215,planetary ionospheres,NaN,uat
500,ionosphere,NaN,agu


In [52]:
pd_terms_total_unique.loc[pd_terms_total_unique['term'].str.contains('geo',case=False)]

,term,definition,source
142,lunar geodesy,NaN,uat
306,lunar geochronology,NaN,uat
393,geocorona,NaN,uat
441,geospatial,NaN,agu
563,mathematical geophysics,NaN,agu
581,geological,NaN,agu
879,geo,Geographic - geocentric corotating - A coordin...,spase
881,geometric factor,A measure of the gathering power of a particl...,spase
1296,geomagnetically induced current (gic),The magnetic fluctuations caused by a geomagne...,nasa


In [53]:
pd_terms_total_unique.to_csv(os.path.join(directory_vocabs,'unique_terms.csv'),index=False)
pd_terms_total_nonunique.to_csv(os.path.join(directory_vocabs,'nonunique_terms.csv'),index=False)
